In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from checklist.perturb import Perturb
from IPython.display import display, HTML
import torch
import spacy
nlp = spacy.load('en_core_web_sm')
cuda_available = torch.cuda.is_available()
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
# bert = ClassificationModel("bert", "pretrained_bert",use_cuda=cuda_available)
dataset = pd.read_csv("data/olid-subset-diagnostic-tests.csv")
dataset

,id,text,labels
0,89200,@USER @USER Who the hell does he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,1
2,55633,"#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!",1
3,16856,@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.,1
4,26072,0-1 lost my acca on the first fucking fight cba,1
5,22067,#Bakersfield is why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!,1
6,60466,"#Christian #America – If we go by #Trump’s example, where liberals support open borders, I guess conservatives support school shootings. Please explain how this makes America great again.",1
7,25685,@USER @USER @USER She is the most disingenuous person in the Senate. If she was my Senator I would hide in shame after her despicable behavior in the SCOTUS hearings.,1
8,16333,#Democrats #Liberals you are being #threatened by #Armed #Nazis and you talk about #disarmament of yourselves and of #Disconnection from the #Military #Lifestyle ? You should #broadcast your #Armory and #killSkills #RightFuckingNOW URL,1
9,78417,"699. Just wanted to tell you you should Hang Out With Me More. i think it’ll be fun! Also, lmao dick for a tongue? So people can easily suck me off. HAHA kidding aside, i aint want a tongue as my dick that looks hella weird.. but my choice was weird to but WHATEVER",1


In [2]:
negated_sentences = pd.DataFrame(columns=['id','text'])
for i in range(100):
    try:
        negated_sentences.at[i,"text"] = Perturb.perturb([nlp(dataset.at[i,"text"])], Perturb.add_negation,keep_original=False).data[0][0]
        negated_sentences.at[i, "id"] = dataset.at[i,"id"]
    except:
        continue
negated_sentences.reset_index(inplace=True,drop=True)
# negated_sentences

In [3]:
# predictions, raw_outputs = bert.predict(negated_sentences["text"].to_list())
predictions = pd.read_csv("data/predicted_negations.csv")["labels"]

In [4]:
negated_sentences["labels"] = predictions
negated_sentences

,id,text,labels
0,89200,@USER @USER Who the hell doesn't he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL,0
2,55633,"#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!",1
3,16856,@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.,0
4,26072,0-1 didn't lose my acca on the first fucking fight cba,1
5,22067,#Bakersfield is not why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!,1
6,60466,"#Christian #America – If we go by #Trump’s example, where liberals support open borders, I don't guess conservatives support school shootings. Please explain how this makes America great again.",0
7,25685,@USER @USER @USER She is not the most disingenuous person in the Senate. If she was my Senator I would hide in shame after her despicable behavior in the SCOTUS hearings.,1
8,16333,#Democrats #Liberals you are being #threatened by #Armed #Nazis and you talk about #disarmament of yourselves and of #Disconnection from the #Military #Lifestyle ? You shouldn't #broadcast your #Armory and #killSkills #RightFuckingNOW URL,0
9,78417,"699. Just didn't want to tell you you should Hang Out With Me More. i think it’ll be fun! Also, lmao dick for a tongue? So people can easily suck me off. HAHA kidding aside, i aint want a tongue as my dick that looks hella weird.. but my choice was weird to but WHATEVER",1


In [8]:
from checklist.editor import Editor
editor = Editor()
masked_hate = editor.template("I hate {mask}")
masked_no_hate = editor.template("I don't hate {mask}")
masked_religion = editor.template("I hate {religion}")
masked_no_religion = editor.template("I don't hate {religion}")
masked_nationality = editor.template("I hate {nationality}")
masked_no_nationality = editor.template("I don't hate {nationality}")

In [16]:
masked_data = masked_hate.data+masked_no_hate.data+masked_religion.data+masked_no_religion.data+masked_nationality.data+masked_no_nationality.data
# masked_data

In [24]:
# masked_predictions, raw_outputs = bert.predict(masked_data)
masked = pd.read_csv("data/predicted_masked.csv",index_col=0)

In [25]:
# masked = pd.DataFrame({'text': masked_data, 'labels': masked_predictions})
masked

,text,labels
0,I hate it,1
1,I hate capitalism,1
2,I hate Trump,1
3,I hate racism,1
4,I hate them,1
5,I hate you,1
6,I hate politics,1
7,I hate this,1
8,I hate that,1
9,I hate him,1
